In [1]:
import numpy as np
import pandas as pd

In [2]:
listings_df = pd.read_csv('listings.csv')
reviews_df = pd.read_csv('reviews.csv')

listings_df.head()

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,6624170,https://www.airbnb.com/rooms/6624170,20240905174946,2024-09-05,previous scrape,"Warm, cozy sunlighted downtown appt",2 room appt. 1.8 km from central station with ...,NaN,https://a0.muscache.com/pictures/df91da10-f7d4...,34670170,...,4.86,4.56,4.65,0363 3CF3 3233 3B24 6BEC,f,1,1,0,0,0.38
1,8837071,https://www.airbnb.com/rooms/8837071,20240905174946,2024-09-05,previous scrape,Cozy apartment in city center,Located in Amsterdam's sweet spot. A stone's t...,see the guide,https://a0.muscache.com/pictures/5fee12d4-61d0...,13034277,...,NaN,NaN,NaN,0363 0C20 1768 FAAA 3556,f,1,1,0,0,NaN
2,716107,https://www.airbnb.com/rooms/716107,20240905174946,2024-09-05,previous scrape,Loft style home nearby city centre,NaN,NaN,https://a0.muscache.com/pictures/9927048/b367a...,3689867,...,4.83,4.66,4.61,0363 0014 1E57 F097 81A5,f,1,1,0,0,1.21
3,664538756986273255,https://www.airbnb.com/rooms/664538756986273255,20240905174946,2024-09-06,previous scrape,Geweldige duurzame eco woonark op unieke plek!,This unique eco houseboat is located in the mo...,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,7647768,...,5.00,5.00,5.00,0363 468D A0A1 6595 5E39,f,1,1,0,0,0.04
4,8191077,https://www.airbnb.com/rooms/8191077,20240905174946,2024-09-05,previous scrape,Old bar apartment,This just renovated apartment for 6 persons is...,Our apartment is situated in the centre of Ams...,https://a0.muscache.com/pictures/miso/Hosting-...,43205475,...,4.79,4.83,4.45,Exempt,f,1,0,1,0,2.41


In [3]:
listings_df.drop(columns = ['description', 'host_about','host_has_profile_pic','neighborhood_overview', 'host_url', 'listing_url', 'scrape_id', 'last_scraped', 'host_thumbnail_url', 'host_picture_url', 'calendar_updated', 'calendar_last_scraped', 'neighbourhood_group_cleansed'], inplace = True)
listings_df['has_license'] = listings_df['license'].notna()
listings_df['host_response_rate'].fillna('%0',inplace = True)
listings_df['host_reponse_rate'] = listings_df['host_response_rate'].str.replace('%','',regex=False).astype(int)
listings_df['price'].fillna('$0', inplace = True)
listings_df['price'] = (listings_df['price'].str.replace('$','',regex = False)).str.replace(',','',regex = False).astype(float).astype(int)
listings_df['amenities_count'] = (listings_df['amenities'].str.replace('[','',regex=False)).str.replace(']','',regex = False).str.split(',').apply(len)
listings_df['has_kitchen'] = listings_df['amenities'].apply(is_kitchen_available)
listings_df['host_is_new'] = listings_df['review_scores_rating'].isna()

listings_df.head()

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,has_license,host_reponse_rate
0,6624170,https://www.airbnb.com/rooms/6624170,20240905174946,2024-09-05,previous scrape,"Warm, cozy sunlighted downtown appt",2 room appt. 1.8 km from central station with ...,NaN,https://a0.muscache.com/pictures/df91da10-f7d4...,34670170,...,4.65,0363 3CF3 3233 3B24 6BEC,f,1,1,0,0,0.38,True,0
1,8837071,https://www.airbnb.com/rooms/8837071,20240905174946,2024-09-05,previous scrape,Cozy apartment in city center,Located in Amsterdam's sweet spot. A stone's t...,see the guide,https://a0.muscache.com/pictures/5fee12d4-61d0...,13034277,...,NaN,0363 0C20 1768 FAAA 3556,f,1,1,0,0,NaN,True,0
2,716107,https://www.airbnb.com/rooms/716107,20240905174946,2024-09-05,previous scrape,Loft style home nearby city centre,NaN,NaN,https://a0.muscache.com/pictures/9927048/b367a...,3689867,...,4.61,0363 0014 1E57 F097 81A5,f,1,1,0,0,1.21,True,100
3,664538756986273255,https://www.airbnb.com/rooms/664538756986273255,20240905174946,2024-09-06,previous scrape,Geweldige duurzame eco woonark op unieke plek!,This unique eco houseboat is located in the mo...,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,7647768,...,5.00,0363 468D A0A1 6595 5E39,f,1,1,0,0,0.04,True,0
4,8191077,https://www.airbnb.com/rooms/8191077,20240905174946,2024-09-05,previous scrape,Old bar apartment,This just renovated apartment for 6 persons is...,Our apartment is situated in the centre of Ams...,https://a0.muscache.com/pictures/miso/Hosting-...,43205475,...,4.45,Exempt,f,1,0,1,0,2.41,True,100


In [24]:
listings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9659 entries, 0 to 9658
Data columns (total 67 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            9659 non-null   int64  
 1   source                                        9659 non-null   object 
 2   name                                          9659 non-null   object 
 3   picture_url                                   9659 non-null   object 
 4   host_id                                       9659 non-null   int64  
 5   host_name                                     9659 non-null   object 
 6   host_since                                    9659 non-null   object 
 7   host_location                                 8547 non-null   object 
 8   host_response_time                            6483 non-null   object 
 9   host_response_rate                            9659 non-null   o

In [27]:
listings_df.to_csv('listings_cleaned.csv')